In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.model_selection import StratifiedKFold

data = pd.read_csv("/content/drive/MyDrive/Creditcard_data.csv")


In [35]:
display(data.head())

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772 entries, 0 to 771
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    772 non-null    int64  
 1   V1      772 non-null    float64
 2   V2      772 non-null    float64
 3   V3      772 non-null    float64
 4   V4      772 non-null    float64
 5   V5      772 non-null    float64
 6   V6      772 non-null    float64
 7   V7      772 non-null    float64
 8   V8      772 non-null    float64
 9   V9      772 non-null    float64
 10  V10     772 non-null    float64
 11  V11     772 non-null    float64
 12  V12     772 non-null    float64
 13  V13     772 non-null    float64
 14  V14     772 non-null    float64
 15  V15     772 non-null    float64
 16  V16     772 non-null    float64
 17  V17     772 non-null    float64
 18  V18     772 non-null    float64
 19  V19     772 non-null    float64
 20  V20     772 non-null    float64
 21  V21     772 non-null    float64
 22  V2

In [37]:
data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [38]:
data.columns


Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [39]:
def run_experiment(sampler, model, X_train, y_train, X_test, y_test):
    X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
    model.fit(X_resampled, y_resampled)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [40]:
X = data.drop('Class', axis=1)
y = data['Class']

In [41]:
random_seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

In [42]:
models = {
    'Random Forest': RandomForestClassifier(random_state=random_seed),
    'Logistic Regression': LogisticRegression(random_state=random_seed),
    'XGBoost': XGBClassifier(random_state=random_seed),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(random_state=random_seed)
}


samplers = {
    'Random OverSampler': RandomOverSampler(random_state=random_seed),
    'SMOTE': SMOTE(random_state=random_seed),
    'Random UnderSampler': RandomUnderSampler(random_state=random_seed),
    'SMOTE-ENN': SMOTEENN(random_state=random_seed),
    'SMOTE-Tomek': SMOTETomek(random_state=random_seed),
}

In [43]:
result_df = pd.DataFrame(columns=['Sampler + Model', 'Accuracy'])

In [44]:
for sampler_name, sampler in samplers.items():
    for model_name, model in models.items():
        accuracy_list = []

        if sampler_name == 'Stratified Sampling':
            skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)
            for train_index, test_index in skf.split(X, y):
                X_train_strat, X_test_strat = X.iloc[train_index], X.iloc[test_index]
                y_train_strat, y_test_strat = y.iloc[train_index], y.iloc[test_index]

                accuracy = run_experiment(sampler, model, X_train_strat, y_train_strat, X_test_strat, y_test_strat)
                accuracy_list.append(accuracy)

        else:
            accuracy = run_experiment(sampler, model, X_train, y_train, X_test, y_test)
            accuracy_list.append(accuracy)

        avg_accuracy = sum(accuracy_list) / len(accuracy_list)
        result_df = result_df.append({'Sampler + Model': f'{sampler_name} + {model_name}', 'Accuracy': avg_accuracy}, ignore_index=True)


<ipython-input-44-cae98722f2d3>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Sampler + Model': f'{sampler_name} + {model_name}', 'Accuracy': avg_accuracy}, ignore_index=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
<ipython-input-44-cae98722f2d3>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result

In [45]:
print(result_df)

                              Sampler + Model  Accuracy
0          Random OverSampler + Random Forest  0.993548
1    Random OverSampler + Logistic Regression  0.877419
2                Random OverSampler + XGBoost  0.993548
3                    Random OverSampler + KNN  0.980645
4                    Random OverSampler + SVM  0.696774
5                       SMOTE + Random Forest  0.993548
6                 SMOTE + Logistic Regression  0.877419
7                             SMOTE + XGBoost  0.980645
8                                 SMOTE + KNN  0.754839
9                                 SMOTE + SVM  0.670968
10        Random UnderSampler + Random Forest  0.709677
11  Random UnderSampler + Logistic Regression  0.593548
12              Random UnderSampler + XGBoost  0.600000
13                  Random UnderSampler + KNN  0.670968
14                  Random UnderSampler + SVM  0.625806
15                  SMOTE-ENN + Random Forest  0.993548
16            SMOTE-ENN + Logistic Regression  0

In [46]:
best_combination = result_df.loc[result_df['Accuracy'].idxmax()]

print("\nBest Combination:")
print(best_combination)


Best Combination:
Sampler + Model    Random OverSampler + Random Forest
Accuracy                                     0.993548
Name: 0, dtype: object
